In [3]:
import numpy as np

In [2]:
N=4
ang = 2*np.pi*np.random.rand(N*2)

## Tequila

In [46]:
H = tq.hamiltonian.QubitHamiltonian.from_string("0.1412*Z(1) + 0.0558*X(0)Z(1)X(2) + \
                                                0.0558*Y(0)Z(1)Y(2) + 0.0868*Z(0)Z(2) + \
                                                0.1425*Z(0)Z(1)Z(2) + 0.1489*Z(1)Z(3) + \
                                                0.0558*X(0)Z(1)X(2)Z(3) + 0.0558*Y(0)Z(1)Y(2)Z(3) + \
                                                0.0868*Z(0)Z(2)Z(3) + 0.1425*Z(0)Z(1)Z(2)Z(3)")

#H = tq.hamiltonian.QubitHamiltonian.from_string("0.0558*X(0)Z(1)X(2) + 0.058*X(0)Z(1)X(2)Z(3)")

In [47]:
A = tq.circuit.QCircuit()
for i in range(N):
    A += tq.circuit.gates.Ry(target=i, angle=ang[i])
A += tq.circuit.gates.CNOT(target=0, control=1)
A += tq.circuit.gates.CNOT(target=1, control=2)
A += tq.circuit.gates.CNOT(target=2, control=3)
for i in range(N):
    A += tq.circuit.gates.Ry(target=i, angle=ang[i+N])

In [48]:
HZ, U = tq.grouping.compile_groups.compile_commuting_parts(H, unitary_circuit="Yen")[0][0]

[+1.0000Z(1), +1.0000Z(0)Z(2), +1.0000Z(3), +1.0000X(0)X(2)]
[+1.0000X(1), +1.0000X(0), +1.0000X(3), +1.0000Z(2)]


In [49]:
A += U
A += tq.circuit.gates.H(target=0)
A += tq.circuit.gates.H(target=1)
A += tq.circuit.gates.H(target=3)

In [50]:
E = tq.objective.ExpectationValue(H=HZ,U=A)#,optimize_measurements=True)
O = tq.simulate(E)
O

-0.06349743922732987

## My Implementation

In [8]:
from measurement_memory import *
from grouping import *

In [9]:
H = [(['IZII', 'XZXI', 'YZYI', 'ZIZI', 'ZZZI', 'IZIZ', 'XZXZ', 'YZYZ', 'ZIZZ', 'ZZZZ'],
     [0.1412, 0.0558, 0.0558, 0.0868, 0.1425, 0.1489, 0.0558, 0.0558, 0.0868, 0.1425])]


# QWC
#H = [(['IZII', 'ZIZI', 'ZZZI', 'IZIZ', 'ZIZZ', 'ZZZZ'], [0.1412, 0.0868, 0.1425, 0.1489, 0.0868,  0.1425]),
#     (['XZXI', 'XZXZ'],[0.0558, 0.0558]), (['YZYI', 'YZYZ'],[0.0558, 0.0558])]

# one QWC group
#H = [(['YZYI', 'YZYZ'],[0.0558, 0.0558])]

In [10]:
def ansatz(x):
    for i in range(N):
        qml.RX(x[i], wires=i)
    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[1,2])
    qml.CNOT(wires=[2,3])
    for i in range(N):
        qml.RX(x[i+N], wires=i)

#def ansatz(x):
#    qml.Hadamard(wires=0)
#    qml.Hadamard(wires=1) 

In [11]:
from Experiment_Molecular import basis_transformation, measurement_rotation

Hg, basis = basis_transformation(H)

In [12]:
#Hg = [(['IXII', 'YXII', 'YIXI', 'IXXI', 'YXXI', 'IXIX', 'YXIX', 'YIXX', 'IXXX', 'YXXX'],
#       [0.1412, 0.0558, -0.0868, 0.0558, -0.1425, 0.1489, 0.0558, -0.0868, 0.0558, -0.1425])]
#basis = [(['IIIZ', 'IZII', 'YIYI', 'XIXI'],['IIIX', 'IXII', 'IIXI', 'YIII'])]

In [13]:
def make_qmlH(Hg:list):
    obs = []
    coeffs = []
    for i in range(len(Hg)):
        for j,p in enumerate(str_to_Pauli(Hg[i][0])):
            obs.append(p)
            coeffs.append(Hg[i][1][j])
    qmlH = qml.Hamiltonian(coeffs, obs)
    return qmlH

In [51]:
dev0 = qml.device("lightning.qubit", wires=N, shots=99999)
@qml.qnode(dev0)
def circuit0(params, T, Q):
    ansatz(params)
    measurement_rotation(T, Q)
    return qml.counts()

def cost0(x):
    E = 0
    for i,h in enumerate(Hg):
        T, Q = basis[i]
        temp = evaluate_eigenstate(circuit0(x, T, Q), h)
        E += temp
    return E

In [52]:
avg0 = []
for _ in range(10):
    avg0.append(cost0(ang))
print(np.mean(avg0))

0.211230078500785


## Pennylane

In [16]:
import pennylane as qml

In [59]:
N=4
#H = qml.Hamiltonian([0.0558, 0.0558],
#                    [qml.PauliX(0)@qml.PauliZ(1)@qml.PauliX(2)@qml.Identity(3), 
#                     qml.PauliX(0)@qml.PauliZ(1)@qml.PauliX(2)@qml.PauliZ(3)])

H = qml.Hamiltonian([0.1412, 0.0558, 0.0558, 0.0868, 0.1425, 0.1489, 0.0558, 0.0558, 0.0868, 0.1425],
                    [qml.PauliZ(1), qml.PauliX(0)@qml.PauliZ(1)@qml.PauliX(2),
                     qml.PauliY(0)@qml.PauliZ(1)@qml.PauliY(2), qml.PauliZ(0)@qml.PauliZ(2),
                     qml.PauliZ(0)@qml.PauliZ(1)@qml.PauliZ(2), qml.PauliZ(1)@qml.PauliZ(3),
                     qml.PauliX(0)@qml.PauliZ(1)@qml.PauliX(2)@qml.PauliZ(3),
                     qml.PauliY(0)@qml.PauliZ(1)@qml.PauliY(2)@qml.PauliZ(3),
                     qml.PauliZ(0)@qml.PauliZ(2)@qml.PauliZ(3),
                     qml.PauliZ(0)@qml.PauliZ(1)@qml.PauliZ(2)@qml.PauliZ(3)])

In [60]:
dev1 = qml.device("lightning.qubit", wires=N)
@qml.qnode(dev1)
def circuit(params):
    ansatz(params)
    return qml.expval(H)

In [61]:
print(qml.draw(circuit)((ang)))

0: ──RX(0.83)─╭●──RX(1.04)─────────────────────┤ ╭<𝓗>
1: ──RX(2.19)─╰X─╭●─────────RX(3.38)───────────┤ ├<𝓗>
2: ──RX(3.90)────╰X────────╭●─────────RX(0.94)─┤ ├<𝓗>
3: ──RX(3.56)──────────────╰X─────────RX(3.61)─┤ ╰<𝓗>


In [62]:
circuit(ang)

array(0.21183147)

In [21]:
H = qml.Hamiltonian([0.0558, 0.0558, 0.0558, 0.1425], 
                    [qml.PauliY(0)@qml.PauliZ(1)@qml.PauliY(2),
                     qml.PauliY(0)@qml.PauliZ(1)@qml.PauliY(2)@qml.PauliZ(3),
                     qml.PauliX(0)@qml.PauliZ(1)@qml.PauliX(2)@qml.PauliZ(3),
                     qml.PauliZ(0)@qml.PauliZ(1)@qml.PauliZ(2)])

H_MM = [(['YZYI', 'YZYZ', 'XZXZ', 'ZZZI'],
         [0.0558, 0.0558, 0.0558, 0.1425])]

In [22]:
Hs = qml.Hamiltonian([-0.0558, -0.0558, 0.0558, 0.1425], 
                    [qml.PauliX(0)@qml.PauliX(1)@qml.PauliZ(2),
                     qml.PauliX(0)@qml.PauliX(1)@qml.PauliZ(2)@qml.PauliX(3),
                                   qml.PauliX(1)@qml.PauliZ(2)@qml.PauliX(3),
                     qml.PauliX(0)@qml.PauliX(1)], grouping_type='QWC')

Hz = qml.Hamiltonian([-0.0558, -0.0558, 0.0558, 0.1425], 
                    [qml.PauliZ(0)@qml.PauliZ(1)@qml.PauliZ(2),
                     qml.PauliZ(0)@qml.PauliZ(1)@qml.PauliZ(2)@qml.PauliZ(3),
                                   qml.PauliZ(1)@qml.PauliZ(2)@qml.PauliZ(3),
                     qml.PauliZ(0)@qml.PauliZ(1)])

Hs_MM = [(['XXZI', 'XXZX', 'IXZX', 'XXII'],
         [-0.0558, -0.0558, 0.0558, 0.1425])]

In [23]:
T, Q = basis[0]

In [24]:
def ansatz2(x):
    for i in range(N):
        qml.RY(x[i], wires=i)
    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[1,2])
    qml.CNOT(wires=[2,3])
    for i in range(N):
        qml.RY(x[i+N], wires=i)

In [25]:
def mr():
    qml.RX(-np.pi/2, wires=1)
    qml.RZ(-np.pi/2, wires=1)
    qml.RX(-np.pi/2, wires=1)
    
    qml.RX(-np.pi/2, wires=0)
    qml.IsingZZ(-np.pi/2, wires=[0,2])
    qml.RX(-np.pi/2, wires=0)
    
    qml.RX(-np.pi/2, wires=3)
    qml.RZ(-np.pi/2, wires=3)
    qml.RX(-np.pi/2, wires=3)
    
    qml.RZ(-np.pi/2, wires=2)
    qml.IsingXX(-np.pi/2, wires=[0,2])
    qml.RZ(-np.pi/2, wires=2)

In [26]:
dev2 = qml.device("lightning.qubit", wires=N, shots=999999)


@qml.qnode(dev2)
def circuit_raw(x):
    ansatz2(x)
    return qml.expval(H)


@qml.qnode(dev2)
def circuit_rot(x):
    ansatz2(x)
    measurement_rotation(T,Q)
    return qml.expval(Hz)


@qml.qnode(dev2)
def circuit_MM(x):
    ansatz2(x)
    measurement_rotation(T,Q)
    return qml.counts()

In [27]:
print(qml.draw(circuit_rot)((ang)))

0: ──RY(0.83)─╭●──RY(1.04)─────────────────────╭RIXII(-1.57)─╭RIZII(-1.57)─╭RIXII(-1.57)
1: ──RY(2.19)─╰X─╭●─────────RY(3.38)───────────├RIXII(-1.57)─├RIZII(-1.57)─├RIXII(-1.57)
2: ──RY(3.90)────╰X────────╭●─────────RY(0.94)─├RIXII(-1.57)─├RIZII(-1.57)─├RIXII(-1.57)
3: ──RY(3.56)──────────────╰X─────────RY(3.61)─╰RIXII(-1.57)─╰RIZII(-1.57)─╰RIXII(-1.57)

──╭RXIII(-1.57)─╭RZIZI(-1.57)─╭RXIII(-1.57)─╭RIIIX(-1.57)─╭RIIIZ(-1.57)─╭RIIIX(-1.57)─╭RIIZI(-1.57)
──├RXIII(-1.57)─├RZIZI(-1.57)─├RXIII(-1.57)─├RIIIX(-1.57)─├RIIIZ(-1.57)─├RIIIX(-1.57)─├RIIZI(-1.57)
──├RXIII(-1.57)─├RZIZI(-1.57)─├RXIII(-1.57)─├RIIIX(-1.57)─├RIIIZ(-1.57)─├RIIIX(-1.57)─├RIIZI(-1.57)
──╰RXIII(-1.57)─╰RZIZI(-1.57)─╰RXIII(-1.57)─╰RIIIX(-1.57)─╰RIIIZ(-1.57)─╰RIIIX(-1.57)─╰RIIZI(-1.57)

──╭RXIXI(-1.57)─╭RIIZI(-1.57)──H─┤ ╭<𝓗>
──├RXIXI(-1.57)─├RIIZI(-1.57)──H─┤ ├<𝓗>
──├RXIXI(-1.57)─├RIIZI(-1.57)────┤ ├<𝓗>
──╰RXIXI(-1.57)─╰RIIZI(-1.57)──H─┤ ╰<𝓗>


In [28]:
circuit_raw(ang)

array(-0.01797958)

In [29]:
circuit_rot(ang)

array(-0.01779635)

In [30]:
evaluate_eigenstate(circuit_MM(ang), Hs_MM[0])

-0.018127079827079826

In [31]:
circuit_MM(ang)

{'0000': 213613,
 '0001': 61280,
 '0010': 31556,
 '0011': 5986,
 '0100': 21050,
 '0101': 104952,
 '0110': 197,
 '0111': 143664,
 '1000': 196760,
 '1001': 85945,
 '1010': 15774,
 '1011': 14113,
 '1100': 33918,
 '1101': 2575,
 '1110': 12643,
 '1111': 55973}

In [32]:
def evaluate_eigenstate(sample, Hg):
    # Input: circuit sample result,
    #        Hg:list of tuple ([Pauli operators], [coefficients])
    # Output : Expactation value <G> evaluated w.r.t. the sample
    total = 0
    G_val = 0
    ops = Hg[0]
    coeffs = Hg[1]
    for k,v in sample.items():
        eigvals = [eigenvalue(c) for c in k]
        kGk = 0
        for i,op in enumerate(ops):
            op_val = 1
            for j, pw in enumerate(op):
                if not pw == 'I':
                    op_val *= eigvals[j]
            kGk += coeffs[i]*op_val
        G_val += kGk*v
        total += v
    return G_val/total

def eigenvalue(state:str):
    return int(1-2*int(state))

In [33]:
T, Q = basis[0]

In [34]:
basis

[(['IZII', 'ZIZI', 'IIIZ', 'XIXI'], ['IXII', 'XIII', 'IIIX', 'IIZI'])]

In [35]:
Hs_MM[0]

(['XXZI', 'XXZX', 'IXZX', 'XXII'], [-0.0558, -0.0558, 0.0558, 0.1425])

In [36]:
from qiskit import QuantumCircuit, QuantumRegister

In [37]:
qr = QuantumRegister(4)
qc = QuantumCircuit(qr)

qc.ry(1.478,0)
qc.rx(0.782,1)
#qc.rx(2.505,2)
#qc.rx(0.000,3)

#qc.cx(0,1)
#qc.cx(0,3)

qc.rx(-np.pi/2, 1)
qc.rz(-np.pi/2, 1)
qc.rx(-np.pi/2, 1)

qc.rx(-np.pi/2, 0)
qc.rzz(-np.pi/2, 0, 2)
qc.rx(-np.pi/2, 0)

qc.rx(-np.pi/2, 3)
qc.rz(-np.pi/2, 3)
qc.rx(-np.pi/2, 3)

qc.rz(-np.pi/2, 0)
qc.rxx(-np.pi/2, 0, 2)
qc.rz(-np.pi/2, 0)

qc.measure_all()

In [38]:
qc.draw()

┌───────────┐┌──────────┐                       ┌──────────┐»
  q0_0: ┤ Ry(1.478) ├┤ Rx(-π/2) ├─────────────■─────────┤ Rx(-π/2) ├»
        ├───────────┤├──────────┤┌──────────┐ │         ├──────────┤»
  q0_1: ┤ Rx(0.782) ├┤ Rx(-π/2) ├┤ Rz(-π/2) ├─┼─────────┤ Rx(-π/2) ├»
        └───────────┘└──────────┘└──────────┘ │ZZ(-π/2) └──────────┘»
  q0_2: ──────────────────────────────────────■─────────────────────»
         ┌──────────┐┌──────────┐┌──────────┐                       »
  q0_3: ─┤ Rx(-π/2) ├┤ Rz(-π/2) ├┤ Rx(-π/2) ├───────────────────────»
         └──────────┘└──────────┘└──────────┘                       »
meas_0: ════════════════════════════════════════════════════════════»
                                                                    »
meas_1: ════════════════════════════════════════════════════════════»
                                                                    »
meas_2: ════════════════════════════════════════════════════════════»
                                                                    »
meas_3: ════════════════════════════════════════════════════════════»
                                                                    »
«        ┌──────────┐┌────────────┐┌──────────┐ ░ ┌─┐         
«  q0_0: ┤ Rz(-π/2) ├┤0           ├┤ Rz(-π/2) ├─░─┤M├─────────
«        └──────────┘│            │└──────────┘ ░ └╥┘┌─┐      
«  q0_1: ────────────┤  Rxx(-π/2) ├─────────────░──╫─┤M├──────
«                    │            │             ░  ║ └╥┘┌─┐   
«  q0_2: ────────────┤1           ├─────────────░──╫──╫─┤M├───
«                    └────────────┘             ░  ║  ║ └╥┘┌─┐
«  q0_3: ───────────────────────────────────────░──╫──╫──╫─┤M├
«                                               ░  ║  ║  ║ └╥┘
«meas_0: ══════════════════════════════════════════╩══╬══╬══╬═
«                                                     ║  ║  ║ 
«meas_1: ═════════════════════════════════════════════╩══╬══╬═
«                                                        ║  ║ 
«meas_2: ════════════════════════════════════════════════╩══╬═
«                                                           ║ 
«meas_3: ═══════════════════════════════════════════════════╩═
«

In [39]:
from qiskit import Aer

#Aer.backends()
simulator = Aer.get_backend('aer_simulator')
result = simulator.run(qc, shots=999999).result()

In [40]:
result.get_counts()

{'0110': 258,
 '1011': 256,
 '1100': 269,
 '0100': 274,
 '1001': 282,
 '0101': 125351,
 '0001': 259,
 '0011': 241,
 '0111': 124431,
 '1101': 124623,
 '1111': 124924,
 '1110': 270,
 '1000': 124656,
 '0010': 124251,
 '0000': 124626,
 '1010': 125028}

In [9]:
def random_Ising_Hamilonian(N):
    coeffs = list(10*np.random.rand((N*(N-1)//2)))
    obs = 
    print(coeffs)
    return 0

In [10]:
H = random_Ising_H(8)

[0.35903335 0.23396331 0.52125575 0.74943571 0.98711329 0.4973292
 0.38641845 0.17679753 0.18872584 0.92022233 0.51327978 0.75254275
 0.54922936 0.55356781 0.81668546 0.31148064 0.47986264 0.28646741
 0.69916638 0.41504747 0.60902894 0.12673642 0.53785501 0.71937774
 0.75233339 0.84003709 0.40225912 0.95467737]


In [16]:
N=8
list(10*np.random.rand((N*(N-1)//2)))

[7.800072029742414,
 3.1344949622085694,
 6.8823107214825185,
 4.8440063617922835,
 9.12932890022304,
 0.6786259271166162,
 5.567027690570455,
 3.819640994927246,
 2.982309324953676,
 1.162077644381534,
 4.4773752442793455,
 3.6705399591611254,
 5.00698932634942,
 9.874604362123037,
 5.465381103635789,
 0.25765790068022376,
 5.3623895839536075,
 5.368671013814162,
 5.550216476263591,
 3.9213179084801055,
 9.321423605876124,
 6.612585459377634,
 7.262301053914064,
 7.09098488547249,
 6.202230120767849,
 8.473124664159883,
 3.0198952394390997,
 9.865358667037956]